# Anomaly Detection | Tools (Assistants API)

## Assistants API / Code Interpreter Tool

The OpenAI Assistants API provides out of the box a tool (Code Interpreter) which allows creating **and** executing of code. This functionality is used to instruct the LLM to create and execute code on the fly in order to analyze the provided time series data.

The [reference](../../TestData/TestData_Reference.txt) time series data and the time series data with [anomalies](../../TestData/TestData_Degradation.txt) is provided to the Assistants API. 



## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed. ([Details and instructions can be found here](../../CreateEnv/CreateEnv.azcli))


## Step 1: Create OpenAI Assistants Client

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. 

In [1]:
#r "nuget: Azure.AI.OpenAI.Assistants, 1.0.0-beta.4"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI.Assistants;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

//configuration file is created during environment creation
static string _configurationFile = @"../../Configuration/application.env";
Env.Load(_configurationFile);

string openAIApiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY") ?? "";
string openAIEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT") ?? "";
string completionDeploymentName = Environment.GetEnvironmentVariable("COMPLETION_DEPLOYMENTNAME") ?? "";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(openAIApiKey);
AssistantsClient assistantsClient = new AssistantsClient(new Uri(openAIEndpoint), azureKeyCredential);


Console.WriteLine($"OpenAI Assistants Client created...");

Installed Packages Azure.AI.OpenAI.Assistants, 1.0.0-beta.4 DotNetEnv, 2.5.0

OpenAI Assistants Client created...


## Step2: Upload files 

File with reference time series data and file with degradation time series data is uploaded to the Assistants API.

In [6]:
using System.IO;

string referenceData = "../../TestData/TestData_Reference.txt";
string degradationData = "../../TestData/TestData_Degradation.txt";

string referenceDataId = "";
string degradationDataId = "";

using (FileStream referenceDataStream = File.OpenRead(referenceData)) {
    Response<OpenAIFile> referenceDataUpload = await assistantsClient.UploadFileAsync(referenceDataStream, OpenAIFilePurpose.Assistants);
    referenceDataId = referenceDataUpload.Value.Id;
    Console.WriteLine($"Reference data uploaded: {referenceDataId}...");

}

using (FileStream degradationDataStream = File.OpenRead(degradationData)) {
    Response<OpenAIFile> degradationDataUpload = await assistantsClient.UploadFileAsync(degradationDataStream, OpenAIFilePurpose.Assistants);
    degradationDataId = degradationDataUpload.Value.Id;
    Console.WriteLine($"Degradation data uploaded: {degradationDataId}...");
}


Reference data uploaded: assistant-AxlepxX6uo0iSEghho2zvFAX...
Degradation data uploaded: assistant-2tbIjtdsOsVtZKjz687O549K...


## Step 3: Create Assistant

A so called Assistant will be created with a system prompt allowing him to write and execute code. The Assistant is created with the two time series data associated to it. The Assistant can also access tools, in this case the Code Interpreter Tool. This is configured within the `AssistantCreationOptions.Tools` collection by providing an Instance of `CodeInterpreterToolDefinition`


In [7]:
//Create Assistant with file reference
string assistantName = "AnomalyDetector";
string assistantInstruction = @"
    You analyze time series data. 
    The time series data is provided to you in text files. 
    Each line in the text file is a valid JSON string. 
    You always get two files. 
    One file contains reference data. 
    The other file contains current time series data. 
    You identify if there's an anomaly in one of the files. 
    You write code to open the files and analyze the files. 
    You execute the code. 
    If there is an anomaly you answer with details about the anomaly.";

string assistantDescription = "Functionality to detect anomalies in Time Series Data. (Tooling (Code generator))";

AssistantCreationOptions assistantCreationOptions; 
assistantCreationOptions = new AssistantCreationOptions(completionDeploymentName){
    Name = assistantName,
    Description = assistantDescription,
    Instructions = assistantInstruction,
}; 
assistantCreationOptions.FileIds.Add(referenceDataId);
assistantCreationOptions.FileIds.Add(degradationDataId);
assistantCreationOptions.Tools.Add(
    new CodeInterpreterToolDefinition()
);

Response<Assistant> responseAssistant = await assistantsClient.CreateAssistantAsync(assistantCreationOptions);
Console.WriteLine($"Assistant created: {responseAssistant.Value.Id}...");


Assistant created: asst_0t25vcry8hrDyLj47L0lULIo...


## Step 4: Create Thread

Create Thread and add message to Thread

In [8]:

//Create Thread
Response<AssistantThread> responseThread = await assistantsClient.CreateThreadAsync();
Console.WriteLine($"Thread created: {responseThread.Value.Id}");

//Add Message to Thread
string message = "Is there an anomaly in the provided file?";
Response<ThreadMessage> responseThreadMessage = await assistantsClient.CreateMessageAsync(
    responseThread.Value.Id, 
    MessageRole.User, 
    message
);

Thread created: thread_ZAKSK7CxGXsdwptqmTF79Uk6


## Step 5: Create Run

Create Run object which allows an Assistant to execute on messages within the Thread object.

In [9]:
//Create Run        
Response<ThreadRun> responseThreadRun = await assistantsClient.CreateRunAsync(
    responseThread.Value.Id, 
    new CreateRunOptions(
        responseAssistant.Value.Id
    ) 
);

Console.WriteLine($"Run created: {responseThreadRun.Value.Id}...");

## Step 6: Wait for completion

Check current progress and get intermediate assistant messages.

In [10]:
do {
            
    //Allow some time for processing
    System.Threading.Thread.Sleep(TimeSpan.FromSeconds(5));
    
    //Get current assistant messages
    Response<PageableList<ThreadMessage>> responseThreadMessages = await assistantsClient.GetMessagesAsync(
        responseThread.Value.Id
    );
    
    foreach(ThreadMessage threadMessage in responseThreadMessages.Value)
    {
        Console.WriteLine($"Message created by/at: {threadMessage.Role} - {threadMessage.CreatedAt}");
        foreach(MessageContent messageContent in threadMessage.ContentItems)
        {
            if (messageContent is MessageTextContent textItem)
                Console.WriteLine($"\t Content: {textItem.Text}");
        }
    }
    responseThreadRun = await assistantsClient.GetRunAsync(responseThread.Value.Id, responseThreadRun.Value.Id);

} while (responseThreadRun.Value.Status != RunStatus.Completed);


Message created by/at: assistant - 5/27/2024 9:31:13 AM +00:00
Message created by/at: assistant - 5/27/2024 9:30:40 AM +00:00
	 Content: To identify if there is an anomaly in one of the provided files, I'll first need to load and examine both text files, which contain lines of valid JSON strings. The first step is to open each file, parse it into a structured format, and then conduct the analysis to identify any anomalies. Let's start by loading the content of both files and examining the structure of the data they contain.
Message created by/at: user - 5/27/2024 9:29:56 AM +00:00
	 Content: Is there an anomaly in the provided file?
Message created by/at: assistant - 5/27/2024 9:31:13 AM +00:00
	 Content: The files have been successfully loaded, and each line contains JSON data with several fields. Here is a sample from each file:

**Reference Data Sample**:
1. `{'DeviceId': 'Compressor1', 'TimeStamp': '10:33:10', 'EnergyConsumption': 0, 'Pressure': 0}`
2. `{'DeviceId': 'Compressor1', 

## Step 7: Get final assistant message

In [12]:
Response<PageableList<ThreadMessage>> responseThreadMessagesFinal = await assistantsClient.GetMessagesAsync(
    responseThread.Value.Id
);

ThreadMessage threadMessageFinal = responseThreadMessagesFinal.Value.First<ThreadMessage>();

Console.WriteLine($"Message created by/at: {threadMessageFinal.Role} - {threadMessageFinal.CreatedAt}");
foreach(MessageContent messageContent in threadMessageFinal.ContentItems)
{
    if (messageContent is MessageTextContent textItem)
        Console.WriteLine($"\t Content: {textItem.Text}");
}

Message created by/at: assistant - 5/27/2024 9:32:55 AM +00:00
	 Content: We have identified several anomalies between the reference and current data files. Here are details about the first few anomalies:

1. At timestamp `10:33:11`, the reference data indicates `Pressure: 1`, whereas the current data indicates `Pressure: 0`.
2. At timestamp `10:33:12`, the reference data indicates `Pressure: 2`, whereas the current data indicates `Pressure: 1`.
3. At timestamp `10:33:13`, the reference data indicates `Pressure: 4`, whereas the current data indicates `Pressure: 2`.
4. At timestamp `10:33:14`, the reference data indicates `Pressure: 4`, whereas the current data indicates `Pressure: 3`.
5. At timestamp `10:33:16`, the reference data indicates `Pressure: 4`, whereas the current data indicates `Pressure: 3`.

In each case, the `Pressure` values in the current data are lower than the corresponding values in the reference data. The `DeviceId`, `TimeStamp`, and `EnergyConsumption` fields rema

## Step 8: Housekeeping

Delete Assistant, Thread and uploaded files with Time Series Data

In [13]:
//Delete Assistant
await assistantsClient.DeleteAssistantAsync(responseAssistant.Value.Id); 

//Delete Thread
await assistantsClient.DeleteThreadAsync(responseThread.Value.Id);

//Delete uploaded files
await assistantsClient.DeleteFileAsync(referenceDataId);
await assistantsClient.DeleteFileAsync(degradationDataId);

Console.WriteLine("Assistant deleted...");
Console.WriteLine("Thread deleted...");
Console.WriteLine("Files deleted...");